In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
# only change the data sets here...
train_path = '../input/train-rg-transformed/train_100.csv'
test_path =  '../input/train-rg-transformed/test_100.csv'
val_path = '../input/train-rg-transformed/val_100.csv'

In [8]:
%%time
test_100 = pd.read_csv(test_path,index_col=0,header=[0,1])
train_100 = pd.read_csv(train_path,index_col=0,header=[0,1])
val_100 = pd.read_csv(val_path,index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 761 ms, sys: 93.1 ms, total: 854 ms
Wall time: 1.19 s


In [9]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [10]:
train_set_100 = pd.concat([train_100,val_100],axis=0)
test_set_100 = pd.concat([val_100,test_100],axis=0)
train_set_100.shape, test_set_100.shape

((1059, 2100), (804, 2100))

In [11]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [12]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [13]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [14]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [15]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [16]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [17]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [18]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [19]:
train_set_100.shape, test_set_100.shape

((1059, 2100), (804, 2100))

In [20]:
assert (train_set_100.columns == test_set_100.columns).all()

In [21]:
t_100 = get_total_abundance(train_set_100)
g_100 = get_grads(train_set_100)
m_100 = get_mean_std(train_set_100)

100%|██████████| 100/100 [00:00<00:00, 233.73it/s]


(1059, 101)


100%|██████████| 100/100 [00:01<00:00, 89.89it/s]


(1059, 2000)


100%|██████████| 100/100 [00:01<00:00, 98.26it/s]

(1059, 2300)


In [22]:
t_100_test = get_total_abundance(test_set_100)
g_100_test = get_grads(test_set_100)
m_100_test = get_mean_std(test_set_100)


100%|██████████| 100/100 [00:00<00:00, 246.57it/s]


(804, 101)


100%|██████████| 100/100 [00:01<00:00, 96.48it/s]


(804, 2000)


100%|██████████| 100/100 [00:00<00:00, 106.81it/s]

(804, 2300)


In [23]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [24]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [25]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [26]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [27]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],1)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_100,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_100.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_100.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.3128921936953116


100%|██████████| 101/101 [00:00<00:00, 364565.15it/s]

11


0.2848754673021992


100%|██████████| 2000/2000 [00:00<00:00, 942434.33it/s]


...

4995:	learn: 0.0042793	total: 1m 22s	remaining: 66.4ms
4999:	learn: 0.0042739	total: 1m 23s	remaining: 0us
fold Loss 17:  0.07139540598163553
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6889169	total: 33.4ms	remaining: 2m 47s
999:	learn: 0.0372268	total: 16.4s	remaining: 1m 5s
1998:	learn: 0.0158054	total: 32.9s	remaining: 49.5s
2997:	learn: 0.0090364	total: 49.4s	remaining: 33s
3996:	learn: 0.0058802	total: 1m 5s	remaining: 16.5s


19it [26:33, 83.71s/it]

4995:	learn: 0.0040398	total: 1m 22s	remaining: 66ms
4999:	learn: 0.0040338	total: 1m 22s	remaining: 0us
fold Loss 18:  0.07812498862400304
---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6890776	total: 31.9ms	remaining: 2m 39s
999:	learn: 0.0338776	total: 16.3s	remaining: 1m 5s
1998:	learn: 0.0144181	total: 32.9s	remaining: 49.4s
2997:	learn: 0.0081091	total: 49.1s	remaining: 32.8s
3996:	learn: 0.0051736	total: 1m 5s	remaining: 16.5s


20it [27:56, 83.80s/it]

4995:	learn: 0.0037196	total: 1m 21s	remaining: 65.6ms
4999:	learn: 0.0037159	total: 1m 21s	remaining: 0us
fold Loss 19:  0.12489629640208214
---------------------------------------------------------------------------
Training Loss: 0.009670831880744185
Loss for sulfide:
0.06767518022912092 0.051596990815316324
AGG Loss: [0.13997203 0.08889311 0.14762875 0.19299808 0.002206   0.14131673
 0.19900921 0.16833287 0.16258852 0.06767518]
AGG Loss mean: 0.13106204692844592


In [28]:
print(np.array(agg_loss).mean())

0.13106204692844592


In [29]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [30]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003193,0.002897,0.015550,0.009789,0.000182,0.005150,0.006472,0.108185,0.004846,0.000816
767,NaN,0.029573,0.008853,0.002287,0.013788,0.000033,0.005546,0.008074,0.098694,0.015351,0.000516
768,NaN,0.892622,0.071202,0.001893,0.060718,0.000182,0.006241,0.817725,0.837792,0.034112,0.000738
769,NaN,0.009336,0.004420,0.047564,0.017432,0.000031,0.971519,0.054724,0.005527,0.964931,0.000534
770,NaN,0.008925,0.006457,0.017386,0.687841,0.000049,0.987220,0.980517,0.004215,0.006898,0.000481
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.008558,0.003480,0.025862,0.024180,0.000095,0.007059,0.008887,0.055664,0.007074,0.000418
1055,NaN,0.007671,0.005273,0.009388,0.025789,0.000545,0.029473,0.011934,0.913986,0.003221,0.000547
1056,NaN,0.003112,0.007354,0.002361,0.013902,0.998160,0.003085,0.006031,0.003393,0.003685,0.000734
1057,NaN,0.004442,0.915269,0.014756,0.052298,0.000207,0.005183,0.044790,0.008478,0.004019,0.001178


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003473,0.004000,0.025439,0.012077,0.000179,0.007853,0.006896,0.463736,0.005134,0.000926
767,NaN,0.059301,0.004705,0.002265,0.016897,0.000042,0.007185,0.013929,0.361482,0.018881,0.000445
768,NaN,0.588891,0.327919,0.001957,0.113194,0.000220,0.007503,0.422215,0.363081,0.139374,0.000604
769,NaN,0.015294,0.004956,0.116428,0.017748,0.000034,0.951300,0.123334,0.006232,0.928230,0.000568
770,NaN,0.008392,0.006979,0.017508,0.152700,0.000035,0.984999,0.974805,0.004703,0.007369,0.000422
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.011520,0.002610,0.015766,0.035431,0.000112,0.008709,0.010923,0.102361,0.010324,0.000366
1055,NaN,0.011608,0.006984,0.009631,0.037024,0.000838,0.079130,0.016259,0.673542,0.003031,0.000525
1056,NaN,0.003509,0.010011,0.002625,0.014719,0.998253,0.003012,0.006678,0.003497,0.003968,0.000822
1057,NaN,0.005368,0.822387,0.016003,0.093122,0.000219,0.008821,0.083688,0.011233,0.004660,0.001082


In [31]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.15317916301918374
carbonate 0.08374686244113039
chloride 0.16148070021994967
iron_oxide 0.24111868049482257
oxalate 0.002181321715375706
oxychlorine 0.1271588770337765
phyllosilicate 0.1859834055784658
silicate 0.15026868116632283
sulfate 0.18690946930410945
sulfide 0.06566847426709663
0.1357695635240233


In [32]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.02627257098589483
carbonate 0.01671332358895014
chloride 0.034156005902406215
iron_oxide 0.05420557137860039
oxalate 0.0005078751690478948
oxychlorine 0.02429283531136425
phyllosilicate 0.04571855431198485
silicate 0.03667478149997074
sulfate 0.02855380279459252
sulfide 0.005526227323935548
0.02726215482667474


In [33]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [34]:
oof_train_df.to_csv('oof_train_100_20fold_5kest.csv', index=False)
oof_test_df.to_csv('oof_test_100_20fold_5kest.csv', index=False)